In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import random
import os

from ssd_model import SSD300, SSD512
from ssd_utils import PriorUtil
from ssd_data import InputGenerator
from ssd_data import preprocess
from ssd_training import SSDLoss
from utils.model import load_weights

In [ ]:
from data_voc import GTUtility
gt_util = GTUtility('data/VOC2007/')

gt_util_train, gt_util_val = gt_util.split(gt_util, split=0.8)

experiment = 'ssd_voc'
num_classes = gt_util.num_classes

#model = SSD300(num_classes=num_classes)
model = SSD512(num_classes=num_classes)

image_size = model.image_size

_, inputs, images, data = gt_util_train.sample_random_batch(batch_size=16, input_size=image_size)

test_idx = 5
test_img = images[test_idx]
test_gt = data[test_idx]

plt.imshow(test_img)
gt_util.plot_gt(test_gt, show_labels=True)
plt.show()

prior_util = PriorUtil(model)

In [ ]:
egt = prior_util.encode(test_gt)
idxs = np.where(np.logical_not(egt[:,4]))[0]
egt[idxs][:,:5]

In [ ]:
#x = prior_util.encode(test_gt)
#y = prior_util.decode(x)


self = prior_util

from utils.vis import to_rec

def plot_assignment(self, map_idx):
    ax = plt.gca()
    im = plt.gci()
    image_height, image_width = image_size = im.get_size()
    
    # ground truth
    boxes = self.gt_boxes
    boxes_x = (boxes[:,0] + boxes[:,2]) / 2. * image_height
    boxes_y = (boxes[:,1] + boxes[:,3]) / 2. * image_width
    for box in boxes:
        xy_rec = to_rec(box[:4], image_size)
        ax.add_patch(plt.Polygon(xy_rec, fill=False, edgecolor='b', linewidth=2))
    plt.plot(boxes_x, boxes_y, 'bo',  markersize=8)
    
    # prior boxes
    for idx, box_idx in self.match_indices.items():
        if idx >= self.map_offsets[map_idx] and idx < self.map_offsets[map_idx+1]:
            x, y = self.priors_xy[idx]
            w, h = self.priors_wh[idx]
            plt.plot(x, y, 'ro',  markersize=4)
            plt.plot([x, boxes_x[box_idx]], [y, boxes_y[box_idx]], '-r', linewidth=1)
            ax.add_patch(plt.Rectangle((x-w/2, y-h/2), w+1, h+1, 
                    fill=False, edgecolor='y', linewidth=2))



from ssd_utils import iou

def encode(self, gt_data, overlap_threshold=0.5, debug=False):
    # calculation is done with normalized sizes

    gt_boxes = self.gt_boxes = np.copy(gt_data[:,:4]) # normalized xmin, ymin, xmax, ymax
    gt_labels = self.gt_labels = np.copy(gt_data[:,4:]) # one_hot classes including background

    num_priors = self.priors.shape[0]
    num_classes = gt_labels.shape[1]

    # TODO: empty ground truth
    if gt_data.shape[0] == 0:
        print('gt_data', type(gt_data), gt_data.shape)

    gt_iou = np.array([iou(b, self.priors_norm) for b in gt_boxes]).T
    
    # assigne gt to priors
    max_idxs = np.argmax(gt_iou, axis=1)
    max_val = gt_iou[np.arange(num_priors), max_idxs]
    prior_mask = max_val > overlap_threshold
    match_indices = max_idxs[prior_mask]

    self.match_indices = dict(zip(list(np.argwhere(prior_mask)[:,0]), list(match_indices)))

    # prior labels
    confidence = np.zeros((num_priors, num_classes))
    confidence[:,0] = 1
    confidence[prior_mask] = gt_labels[match_indices]

    # compute local offsets from ground truth boxes
    gt_xy = (gt_boxes[:,2:4] + gt_boxes[:,0:2]) / 2.
    gt_wh = gt_boxes[:,2:4] - gt_boxes[:,0:2]
    gt_xy = gt_xy[match_indices]
    gt_wh = gt_wh[match_indices]
    priors_xy = self.priors_xy[prior_mask] / self.image_size
    priors_wh = self.priors_wh[prior_mask] / self.image_size
    offsets = np.zeros((num_priors, 4))
    offsets[prior_mask, 0:2] = (gt_xy - priors_xy) / priors_wh
    offsets[prior_mask, 2:4] = np.log(gt_wh / priors_wh)
    offsets[prior_mask, :] /= self.priors[prior_mask,-4:] # variances

    return np.concatenate([offsets, confidence], axis=1)
    
def decode(self, model_output, confidence_threshold=0.01, keep_top_k=200):
    # calculation is done with normalized sizes

    offsets = model_output[:,:4]
    confidence = model_output[:,4:]

    num_priors = offsets.shape[0]
    num_classes = confidence.shape[1]

    priors_xy = self.priors_xy / self.image_size
    priors_wh = self.priors_wh / self.image_size

    # compute bounding boxes from local offsets
    boxes = np.empty((num_priors, 4))
    offsets *= self.priors[:,-4:] # variances
    boxes_xy = priors_xy + offsets[:,0:2] * priors_wh
    boxes_wh = priors_wh * np.exp(offsets[:,2:4])
    boxes[:,0:2] = boxes_xy - boxes_wh / 2. # xmin, ymin
    boxes[:,2:4] = boxes_xy + boxes_wh / 2. # xmax, ymax
    boxes = np.clip(boxes, 0.0, 1.0)

    prior_mask = confidence > confidence_threshold

    # TODO: number of confident boxes, compute bounding boxes only for those?
    #print(np.sum(np.any(prior_mask[:,1:], axis=1)))

    # do non maximum suppression
    results = []
    for c in range(1, num_classes):
        mask = prior_mask[:,c]
        boxes_to_process = boxes[mask]
        if len(boxes_to_process) > 0:
            confs_to_process = confidence[mask, c]
            feed_dict = {
                self.boxes: boxes_to_process,
                self.scores: confs_to_process
            }
            idx = self.sess.run(self.nms, feed_dict=feed_dict)
            good_boxes = boxes_to_process[idx]
            good_confs = confs_to_process[idx][:, None]
            labels = np.ones((len(idx),1)) * c
            c_pred = np.concatenate((good_boxes, good_confs, labels), axis=1)
            results.extend(c_pred)
    results = np.array(results)
    if len(results) > 0:
        order = np.argsort(results[:, 1])[::-1]
        results = results[order]
        results = results[:keep_top_k]
    self.results = results
    return results

x = encode(self, test_gt)
y = decode(self, x)


#for idx in range(len(prior_util.prior_maps)):
for idx in [1,2,3,4]:
    m = prior_util.prior_maps[idx]
    plt.figure(figsize=[10]*2)
    plt.imshow(test_img)
    m.plot_locations()
    #m.plot_boxes([0, 10, 100])
    #gt_util.plot_gt(test_gt)
    plot_assignment(self, idx)
    prior_util.plot_results(y, show_labels=False)
    plt.show()

In [ ]:
enc = prior_util.encode(test_gt)
prior_util.decode(enc)[:,:4]

In [ ]:
test_gt[:,:4]

In [ ]:
# plot ground truth
for i in range(4):
    plt.figure(figsize=[8]*2)
    plt.imshow(images[i])
    gt_util.plot_gt(data[i])
    plt.show()

In [ ]:
# plot prior boxes
for i, m in enumerate(prior_util.prior_maps):
    plt.figure(figsize=[8]*2)
    #plt.imshow(images[7])
    plt.imshow(images[8])
    m.plot_locations()
    #m.plot_boxes([0, 10, 100])
    m.plot_boxes([0])
    plt.axis('off')
    #plt.savefig('plots/ssd_priorboxes_%i.pgf' % (i), bbox_inches='tight')
    #print(m.map_size)
    plt.show()

In [ ]:
for l in model.layers:
    try:
        ks = l.weights[0].shape
    except:
        ks = ""
    print("%30s %16s %24s %20s" % (l.name, l.__class__.__name__, l.output_shape, ks))

In [ ]:
model.summary()

In [ ]:
batch_size = 24

gen = InputGenerator(gt_util_train, prior_util, batch_size, model.image_size, 
        augmentation=False,
        vflip_prob=0.0, do_crop=False)

In [ ]:
g = gen.generate()

In [ ]:
%%time
for j in range(3000):
    i, o = next(g)
    print(j, len(i))
    if len(i) != batch_size:
        print('FOOOOOOOOOOO')

In [ ]:
import cProfile

g = gen.generate()

p = cProfile.Profile()

p.enable()

i, o = next(g)

p.disable()

p.print_stats(sort='cumulative')